In [1]:
from  scipy import ndimage
import math
import numpy as np
import argparse
import cv2
from math import atan2, pi
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from keras.models import Sequential, load_model
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
from nltk.tag import pos_tag
import json
#from fuzzywuzzy import fuzz

c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imageFilePath =r'C:\Users\sachin\Desktop\f1.png'

KERAS_Alphabet_model_File_Path = r'C:\Users\sachin\Desktop\Keras_Alphabet_Model.h5'
Alphabet_Mapping_List = list(string.ascii_uppercase)
kernel = np.ones((2,2),np.uint8)
index = 0
global counts,X,Y,W,H

In [3]:
KERAS_Alphabet_Model = load_model(KERAS_Alphabet_model_File_Path)

In [14]:
def Get_Processed_Image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray_image,(3,3),0)
    ret3,Otsu_Threshold_GaussianBlur_Image = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    Otsu_Threshold_GaussianBlur_Image = cv2.bitwise_not(Otsu_Threshold_GaussianBlur_Image)

    closing = cv2.morphologyEx(Otsu_Threshold_GaussianBlur_Image, cv2.MORPH_CLOSE, kernel)
    #cv2.imwrite(r'C:\Users\sachin\Desktop\closing.jpg',closing)

    return closing



def Get_Dilated_Image(image, number):
    global index
    kernel = np.ones((3,number),np.uint8)
    img_dilation = cv2.dilate(image, kernel, iterations=3)
    #index = index + 1
    #cv2.imwrite(r'C:\Users\sachin\Desktop\img_dilation_'+str(index)+'.jpg',img_dilation)
    return img_dilation
        

def Get_Countours(input_Image):
    temp_image, contours, hierarchy = cv2.findContours(input_Image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours


def Sort_Countours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b:b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return (cnts)

def Draw_Contours(Image, Contours):
    for cnt in Contours:
        x,y,w,h = cv2.boundingRect(cnt)
        #print (str(x) + " - " + str(y)+ " - " + str(w) + " - " + str(h))
        cv2.rectangle(Image,(x,y),(x+w,y+h),(0,255,255),1)
    #cv2.imwrite(r'C:\Users\sachin\Desktop\croppedrect.jpg',Image)   




def getNewResizedImage(input_Image, Image_Size):
    height,width = input_Image.shape
    #print (height, width)

    if width > height:
        aspect_Ratio = (float)(width/height)
        width = 20
        height = round(width/aspect_Ratio)
    else:
        aspect_Ratio = (float)(height/width)
        height = 20
        width = round(height/aspect_Ratio)
        
    input_Image = cv2.resize(input_Image, (width,height), interpolation = cv2.INTER_AREA )
    
    height,width = input_Image.shape
    
    number_Of_Column_To_Add = 28-width
    temp_Column = np.zeros( (height , int(number_Of_Column_To_Add/2)), dtype = np.uint8)
    input_Image = np.append(temp_Column, input_Image, axis=1)
    input_Image = np.append(input_Image, temp_Column, axis=1)


    height,width = input_Image.shape

    number_Of_Row_To_Add = 28-height
    temp_Row= np.zeros( (int(number_Of_Row_To_Add/2) , width ), dtype = np.uint8)
    input_Image = np.concatenate((temp_Row,input_Image))
    input_Image = np.concatenate((input_Image,temp_Row))

    return cv2.resize(input_Image, (Image_Size,Image_Size), interpolation = cv2.INTER_AREA )


def Get_Average_Space_From_Image_Line(contours):
    total_space = 0
    number_Of_Character = 0
    last_Point = -1
    avg_Space = 0
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        #print (x,y,w,h)

        if w > 30 and h > 30 and x > 0 and y > 0:
            if last_Point == -1:
                last_Point = 0
            else:
                total_space = total_space + ( x - last_Point )
            last_Point = x + w

            number_Of_Character = number_Of_Character + 1

    avg_Space = int (total_space/number_Of_Character)

    total_space = 0
    last_Point = -1
    number_Of_Character = 0

##    for cnt in contours:
##        x,y,w,h = cv2.boundingRect(cnt)
##
##        if w > 30 and h > 30 and x > 0 and y > 0:
##            if last_Point == -1:
##                last_Point = 0
##            else:
##                space = ( (x + w) - last_Point )
##                if space < avg_Space:
##                    total_space = total_space + space
##                    number_Of_Character = number_Of_Character + 1
##            last_Point = x + w
##            
##    avg_Space = int (total_space/number_Of_Character)

    return avg_Space







def Get_Text_From_Image(Image, contours):
    global count,X,Y,W,H, list_Character_Positions
    KERAS_Alphabet_Prediction = ''

    total_space = 0
    number_Of_Character = 0
    last_Point = -1

    #avg_Space = Get_Average_Space_From_Image_Line(contours)
    last_Point = -1
    total_space = 0
    #print (avg_Space)
    
    Word_Dilated_Image = Get_Dilated_Image(Image,10)
    Word_Contours = Get_Countours(Word_Dilated_Image)
    Word_Contours = Sort_Countours(Word_Contours,"left-to-right")
    last_Word_Contour_Index = 0
    Word_X, Word_Y, Word_W, Word_H = cv2.boundingRect(Word_Contours[last_Word_Contour_Index])
    last_Word_Contour_Max_X_Range = Word_X + Word_W
    # i.e X + W
            

    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        give_Space = False
        #print (x,y,w,h)

        # Reject if Contour is not of desired size (too small)
        if w > 8 and h > 15 and x > 0 and y > 0:

#             ## Spacing based on average space between character
#             if last_Point == -1:
#                 last_Point = 0
#             else:
#                 total_space = ( x - last_Point )

#             last_Point = x + w

#             if total_space > (avg_Space*1.3):
#                 give_Space = True
            
            ## Spacing based on word formation
            if x > last_Word_Contour_Max_X_Range:
                give_Space = True
                last_Word_Contour_Index = last_Word_Contour_Index + 1
                Word_X, Word_Y, Word_W, Word_H = cv2.boundingRect(Word_Contours[last_Word_Contour_Index])
                last_Word_Contour_Max_X_Range = Word_X + Word_W
                # i.e X + W
            
    
                
            if give_Space == True:
                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + " "
                list_Character_Positions.append((-1,-1,-1,-1," "))
                

            resize_image = getNewResizedImage(Image[y:y+h, x:x+w] , 28)
            path = r'C:\Users\sachin\Desktop\Images\\' + str(count)+'.png'
            cv2.imwrite(path,resize_image)
            resize_image = resize_image.flatten()
            count = count + 1

            temp_Index = int(KERAS_Alphabet_Model.predict_classes(resize_image.reshape(1,784)/255.0)[0])
            alphabet_probability = (KERAS_Alphabet_Model.predict_proba(resize_image.reshape(1,784)/255.0))
            sort_alphabet_probability = -np.sort(-alphabet_probability)

            KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + Alphabet_Mapping_List[int(temp_Index)]

            list_Character_Positions.append((x+X,y+Y,w,h,str(Alphabet_Mapping_List[int(temp_Index)])))

##            if sort_alphabet_probability[0,0] > 0.99:
##                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + Alphabet_Mapping_List[int(temp_Index)]
##            else:
##                alternate_Probable_Alphabet = Alphabet_Mapping_List[int(np.where(alphabet_probability == sort_alphabet_probability[0,1])[1][0])]
##                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + "[" + Alphabet_Mapping_List[int(temp_Index)] + "," + alternate_Probable_Alphabet + "]"

    list_Character_Positions.append((-1,-1,-1,-1," "))

    return KERAS_Alphabet_Prediction

In [15]:
image = cv2.imread(imageFilePath)
image = Get_Processed_Image(image)
image_With_Lines = Get_Dilated_Image(image, 50)

contours = Get_Countours(image_With_Lines)
contours = Sort_Countours(contours, "top-to-bottom")

Draw_Contours(cv2.imread(imageFilePath),contours)
Predicted_Text = ''
temp_Index = ''
global list_Character_Positions
X=Y=W=H=0
count = 20
list_Character_Positions = []

In [16]:
counts = 0
index = 0
Predicted_Text = ''
for line_Area in contours:
    counts = counts + 1
    x,y,w,h = cv2.boundingRect(line_Area)
    X,Y,W,H = x,y,w,h
    line_Image = image[y:y+h, x:x+w]
    path = r'C:\Users\sachin\Desktop\Images\\' + str(counts)+'.png'
    cv2.imwrite(path,line_Image)

    line_Contours = Get_Countours(line_Image)
    line_Contours = Sort_Countours(line_Contours,"left-to-right")
    #Draw_Contours(line_Image,line_Contours)
    
    text = (Get_Text_From_Image(line_Image, line_Contours))
    print (text)

    Predicted_Text = Predicted_Text + " " + text


print('KERAS_Alphabet_Predict \n' + Predicted_Text)

AMMEUJJAYALAJXN
H
MM ROHANEJ
ML VUYLSOBGYML
CO
MB MORPET
M PUNE
M MH
DPM BDGLZBA
MR OZX ZGSAAN
OT SJOZYS JM JM
TTTTTTTTTTTTT
AWIMLOJM
A
KERAS_Alphabet_Predict 
 AMMEUJJAYALAJXN H MM ROHANEJ ML VUYLSOBGYML CO MB MORPET M PUNE M MH DPM BDGLZBA MR OZX ZGSAAN OT SJOZYS JM JM TTTTTTTTTTTTT AWIMLOJM A
